In [65]:
import joblib
import numpy as np
import pandas as pd
import pycountry
import sys

sys.path.append("..") # Find NN module

In [66]:
# Load neural network model and scalers/encoders
model = joblib.load("../trained_models/neural_network.pkl")
countries = joblib.load("../trained_models/countries.pkl")
country_encoder = joblib.load("../trained_models/country_encoder.pkl")

In [67]:
# Forecast until 2050 for all countries
df = None
for year in range(2023, 2051):
    c = country_encoder.transform(np.array(countries))
    prediction = model.predict(pd.DataFrame.from_dict({"country_encoded": c, "year": [year]*len(countries)}).astype("float32"))
    if df is None:
        df = pd.DataFrame({"country": countries, "year": [year]*len(countries), "TOE_HAB": prediction})
    else:
        df = pd.concat([df, pd.DataFrame({"country": countries, "year": [year]*len(countries), "TOE_HAB": prediction})], axis=0)
df.head(5)

,country,year,TOE_HAB
0,AL,2023,0.700458
1,AT,2023,2.796807
2,BE,2023,2.932049
3,BG,2023,1.454154
4,CY,2023,1.971355


In [68]:
# Change from ISO-2 country codes to country names
def get_country_name(country):
    values = pycountry.countries.get(alpha_2=country)
    if values is None:
        return None
    else:
        return values.name


df["country"] = df["country"].apply(lambda x: get_country_name(x))
df.head(5)

,country,year,TOE_HAB
0,Albania,2023,0.700458
1,Austria,2023,2.796807
2,Belgium,2023,2.932049
3,Bulgaria,2023,1.454154
4,Cyprus,2023,1.971355


In [69]:
# Drop nulls
df = df.dropna()
df.head(5)

,country,year,TOE_HAB
0,Albania,2023,0.700458
1,Austria,2023,2.796807
2,Belgium,2023,2.932049
3,Bulgaria,2023,1.454154
4,Cyprus,2023,1.971355


In [81]:
gt_df = pd.read_csv("energy_consumption_eu.csv").drop(columns=["MTOE"])
gt_df["type"] = ["Dataset"]*len(gt_df)
df["type"] = ["Predicted"]*len(df)
df = df.merge(gt_df, how="outer")
df.head()

,country,year,TOE_HAB,type
0,Albania,2000,0.50,Dataset
1,Albania,2001,0.51,Dataset
2,Albania,2002,0.59,Dataset
3,Albania,2003,0.58,Dataset
4,Albania,2004,0.65,Dataset


In [ ]:
# Save predictions
df.to_csv("toe_hab_pred.csv", index=False)